# Support Vector Machine

In [2]:
import numpy as np
import pandas as pd
#import sklearn as sm
from sklearn import svm
import math

## Data - Response

In [3]:
PROFILE = pd.read_csv('/Users/anrismith/Downloads/DSIDE_HomeWork/PROFILE.csv')
PROFILE#(933, 7)

,Unnamed: 0,Unnamed: 0.1,mun_code,Efficient_measure,wellfare_measure,profile
0,0,2013,BUF,0.998145,0.574704,1
1,1,2013,CPT,0.999998,0.708696,1
2,2,2013,DC1,1.000000,0.712714,1
3,3,2013,DC10,0.996483,0.661456,1
4,4,2013,DC12,0.994372,0.258010,2
5,5,2013,DC13,0.997008,0.382382,2
6,6,2013,DC14,0.998879,0.337836,2
7,7,2013,DC15,0.987575,0.173731,2
8,8,2013,DC16,0.978450,0.682325,1
9,9,2013,DC18,0.000000,0.684813,4


In [107]:
#calculate svm output across all years and by year.
#to start calculate prediction for 2015

## Youth - Predictors

Predictor explanations:
Tertiary_prop - Proportion of youth aged between 20 and 24 that have any tertiary education.
Given_Birth - proportion of females between ages 15 and 24 that have given birth
EmpAdult_Prop - proportion of homes that have at least one employed adult in the house.
Citizenship_Prop - proportion of people that are citizens.
Educ_Attend_Prop - porportion of youth aged between 15 and 24 that attend school.
Female_prop - proportion of females.
Overcrowded_Prop - proportion of homes that are overcrowded.
IncPoverty_Prop - proportion of people classified as poor based on income
'Living_with_Parents_Prop - proportion of youth living with both mother and father.

In [52]:
tertiary_educ = pd.read_csv("/home/dside/Documents/GitHub/Youth data/YOUTH_AGE_20_TO_24_EDUCATION_LEVEL_South_Africa.csv")
tertiary_educ['Tertiary_prop'] = tertiary_educ['Any tertiary'] / tertiary_educ['total']

Given_Birth = pd.read_csv("/home/dside/Documents/GitHub/Youth data/YOUTH_FEMALE_GIVEN_BIRTH_AGE_South_Africa.csv")
Given_Birth['birth_prop'] = Given_Birth['Given birth'] / Given_Birth['total']
Given_Birth = Given_Birth[['geo_code', 'birth_prop']]
Given_Birth = Given_Birth.rename(columns = {'geo_code': 'mun_code'})

Emp_Adult = pd.read_csv("/home/dside/Documents/GitHub/Youth data/YOUTH_HOUSEHOLD_EMPLOYMENT_South_Africa.csv")
Emp_Adult['EmpAdult_Prop'] = Emp_Adult['At least one employed adult'] / Emp_Adult['total']
Emp_Adult = Emp_Adult[['geo_code', 'EmpAdult_Prop']]
Emp_Adult  = Emp_Adult.rename(columns = {'geo_code': 'mun_code'})

Citizenship = pd.read_csv("/home/dside/Documents/GitHub/Youth data/YOUTH_CITIZENSHIP_South_Africa.csv")
Citizenship['Citizenship_Prop'] = Citizenship['Yes'] / Citizenship['total']
Citizenship = Citizenship[['geo_code', 'Citizenship_Prop']]
Citizenship = Citizenship.rename(columns = {'geo_code': 'mun_code'})

Educ_Attend = pd.read_csv("/home/dside/Documents/GitHub/Youth data/YOUTH_EDUCATION_ATTENDANCE_AGE_INCOMPLETED_YEARS_GENDER_South_Africa.csv")
Educ_Attend['EducAttend_Prop'] = Educ_Attend['Yes'] / Educ_Attend['total']
Educ_Attend = Educ_Attend[['geo_code', 'EducAttend_Prop']]
Educ_Attend = Educ_Attend.rename(columns = {'geo_code': 'mun_code'})

Gender = pd.read_csv("/home/dside/Documents/GitHub/Youth data/YOUTH_GENDER_POPULATION_GROUP_South_Africa.csv")
Gender['Female_Prop'] = Gender['Female'] / Gender['total']
Gender = Gender[['geo_code', 'Female_Prop']]
Gender = Gender.rename(columns = {'geo_code': 'mun_code'})

Crowding = pd.read_csv("/home/dside/Documents/GitHub/Youth data/YOUTH_HOUSEHOLD_CROWDED_GENDER_South_Africa.csv")
Crowding['Overcrowded_Prop'] = Crowding['Overcrowded'] /Crowding['total']
Crowding = Crowding[['geo_code', 'Overcrowded_Prop']]
Crowding = Crowding.rename(columns = {'geo_code': 'mun_code'})

Inc_Poverty =  pd.read_csv("/home/dside/Documents/GitHub/Youth data/YOUTH_INCOME_POVERTY_AGE_GROUP_South_Africa.csv")
Inc_Poverty['IncPoverty_Prop'] = Inc_Poverty['Income-poor'] / Inc_Poverty['total']
Inc_Poverty = Inc_Poverty[['geo_code', 'IncPoverty_Prop']]
Inc_Poverty = Inc_Poverty.rename(columns = {'geo_code': 'mun_code'})

Living_with_Parents =  pd.read_csv("/home/dside/Documents/GitHub/Youth data/YOUTH_LIVING_WITH_PARENTS_GENDER_South_Africa.csv")
Living_with_Parents['Living_with_Parents_Prop'] = Living_with_Parents['Both parents'] / Living_with_Parents['total']
Living_with_Parents = Living_with_Parents[['geo_code', 'Living_with_Parents_Prop']]
Living_with_Parents = Living_with_Parents.rename(columns = {'geo_code': 'mun_code'})


# ADD IN MORE YOUTH PREDICTORS AS WELL AS THE FINANCIAL MEASURES.

## Financial predictors 

In [53]:
Fin_measures = pd.read_csv('/home/dside/Documents/GitHub/MunicipalData/fin_measures.csv')
#del Fin_measures['Unnamed: 0']
Fin_measures # (713, 12)

,Unnamed: 0,mun_code,repairs_PPE,mun_name,currRatio,cash_costCoverage,OppSurplusMargin,debt_Rev,Efficient_spending
0,2013,BUF,0.025328,Buffalo City,2.033938,0.511104,0.110572,0.090928,0.964653
1,2013,DC1,0.073776,West Coast,4.220636,0.099576,0.022806,0.116719,1.000000
2,2013,DC10,0.007526,Cacadu,7.712887,1.927764,-0.048996,0.000000,0.932971
3,2013,DC12,0.000000,Amathole,3.203076,0.284329,0.281732,-0.042301,0.892734
4,2013,DC13,0.000611,Chris Hani,2.247719,0.110483,0.196916,0.186563,0.942970
5,2013,DC14,0.000000,Joe Gqabi,1.244426,0.047982,0.105474,0.093837,0.978627
6,2013,DC15,0.026370,O.R.Tambo,1.123713,-0.009961,0.220736,0.127911,0.763193
7,2013,DC16,0.063304,Xhariep,0.382010,-0.193086,-0.796242,0.146114,0.589258
8,2013,DC18,0.006782,Lejweleputswa,4.045375,0.247428,0.030179,0.077629,-18.059578
9,2013,DC19,0.000000,Thabo Mofutsanyane,1.594245,0.015990,-0.293931,0.294425,0.573724


In [60]:
#Inc_Poverty

In [48]:
svm_predictors = pd.DataFrame()
svm_predictors['mun_code'] = tertiary_educ['geo_code']
svm_predictors['Tertiary_educ'] = tertiary_educ['Tertiary_prop']

svm_predictors = svm_predictors.merge(Given_Birth, on = ['mun_code'])
svm_predictors = svm_predictors.merge(Emp_Adult, on = ['mun_code'])
svm_predictors = svm_predictors.merge(Citizenship, on = ['mun_code'])
svm_predictors = svm_predictors.merge(Educ_Attend, on = ['mun_code'])
svm_predictors = svm_predictors.merge(Gender, on = ['mun_code'])
svm_predictors = svm_predictors.merge(Crowding, on = ['mun_code'])
svm_predictors = svm_predictors.merge(Inc_Poverty, on = ['mun_code'])
svm_predictors = svm_predictors.merge(Living_with_Parents , on = ['mun_code'])

#introducing financial measures - remeber this has a time element.
svm_predictors = Fin_measures.merge(svm_predictors , on = ['mun_code'], how = 'left')
del svm_predictors['mun_name']

svm_predictors = svm_predictors.drop_duplicates() #(1305, 21)
# 713 rows - matches Fin Measures length
svm_predictors = svm_predictors.dropna() # when the youth data is updated this wont be neccessary
svm_predictors #(491, 21)


,Unnamed: 0,mun_code,repairs_PPE,currRatio,cash_costCoverage,OppSurplusMargin,debt_Rev,Efficient_spending,Tertiary_educ,birth_prop,EmpAdult_Prop,Citizenship_Prop,EducAttend_Prop,Female_Prop,Overcrowded_Prop,IncPoverty_Prop,Living_with_Parents_Prop
0,2013,BUF,0.025328,2.033938,0.511104,0.110572,0.090928,0.964653,0.069307,0.249921,0.586196,0.985233,0.572181,0.508867,0.198961,0.556065,0.215037
44,2013,EC103,0.011583,0.211702,-0.018140,-0.043160,0.111912,0.556501,0.017632,0.353147,0.652574,0.983153,0.398154,0.516187,0.104969,0.693819,0.227746
46,2013,EC104,0.012718,1.205775,-0.107639,0.023174,0.080389,0.469701,0.117075,0.183982,0.606591,0.974696,0.534894,0.521548,0.155457,0.559891,0.227053
48,2013,EC106,0.029686,0.978935,0.085740,0.073931,0.129860,0.728742,0.015456,0.340372,0.695891,0.961879,0.331143,0.457796,0.131357,0.635276,0.282877
50,2013,EC107,0.013742,0.170533,0.002842,0.060959,0.183974,0.792633,0.013453,0.396996,0.606949,0.991768,0.328194,0.487126,0.174465,0.677640,0.254633
52,2013,EC108,0.007734,0.541311,0.037559,-0.092370,0.213519,0.922728,0.029454,0.373232,0.768660,0.977036,0.345957,0.499622,0.136771,0.530460,0.344163
54,2013,EC109,0.004275,0.938684,0.002210,-0.258410,0.058123,1.000000,0.014294,0.426052,0.851573,0.968225,0.266562,0.483269,0.242921,0.573275,0.295050
56,2013,EC121,0.000000,3.656777,0.283508,0.203896,0.052712,0.672913,0.021041,0.287403,0.210729,0.993105,0.653823,0.498808,0.570497,0.849304,0.153586
58,2013,EC122,0.009135,1.732011,0.066906,0.200743,0.029793,0.527772,0.039098,0.280292,0.276225,0.992682,0.667079,0.490003,0.401260,0.787431,0.154110
60,2013,EC123,0.000000,6.131920,0.039118,-0.137913,0.109436,-0.507262,0.015898,0.253555,0.449249,0.991599,0.534223,0.471351,0.513330,0.751377,0.178176


In [43]:
# SVM PREDICTORS OVER YEARS
svm_predictors.to_csv('/home/dside/Documents/GitHub/MunicipalData/svm_predictore.csv')

## Average Measures 

In [57]:
svm_avg_measures = pd.read_csv('/home/dside/Documents/GitHub/Analysis/svm_avg_predictors.csv')
svm_avg_measures

,mun_code,repairs_PPE,currRatio,cash_costCoverage,OppSurplusMargin,debt_Rev,Efficient_spending,Tertiary_educ,birth_prop,EmpAdult_Prop,Citizenship_Prop,EducAttend_Prop,Female_Prop,Overcrowded_Prop,IncPoverty_Prop,Living_with_Parents_Prop
0,BUF,0.023720,2.482919,0.471867,0.100104,0.101412,0.899330,0.069307,0.249921,0.586196,0.985233,0.572181,0.508867,0.198961,0.556065,0.215037
1,CPT,0.000000,1.266176,0.123189,0.073212,0.028515,0.999982,0.089004,0.259899,0.785328,0.954220,0.460475,0.510679,0.135318,0.373324,0.353355
2,EC101,0.000000,1.729292,0.068318,-0.100950,0.039687,0.918711,0.036421,0.309103,0.685474,0.992386,0.424163,0.488112,0.182157,0.615141,0.248494
3,EC102,0.000000,1.228257,0.014269,-0.075396,0.072021,0.791688,0.023654,0.310874,0.516252,0.989414,0.398775,0.504205,0.129970,0.659468,0.236147
4,EC103,0.011583,0.211702,-0.018140,-0.043160,0.111912,0.556501,0.017632,0.353147,0.652574,0.983153,0.398154,0.516187,0.104969,0.693819,0.227746
5,EC104,0.012702,1.280449,-0.120816,0.033512,0.040195,0.553571,0.117075,0.183982,0.606591,0.974696,0.534894,0.521548,0.155457,0.559891,0.227053
6,EC105,0.000000,0.881612,0.000194,0.077780,0.470084,0.230729,0.026350,0.264853,0.599131,0.979132,0.435303,0.494508,0.150628,0.619292,0.237644
7,EC106,0.014843,0.899184,0.147281,0.088517,0.130456,0.641236,0.015456,0.340372,0.695891,0.961879,0.331143,0.457796,0.131357,0.635276,0.282877
8,EC107,0.008665,0.196663,0.005120,0.139632,0.216102,0.878345,0.013453,0.396996,0.606949,0.991768,0.328194,0.487126,0.174465,0.677640,0.254633
9,EC108,0.002578,0.601332,0.062942,-0.024470,0.176019,0.957415,0.029454,0.373232,0.768660,0.977036,0.345957,0.499622,0.136771,0.530460,0.344163


In [59]:
avg_profiles = pd.read_csv('/home/dside/Documents/GitHub/Analysis/avg_PROFILE.csv')
avg_profiles

,mun_code,Efficient_measure,wellfare_measure,avg_profile
0,BUF,1.060883,0.355695,1
1,CPT,1.072637,1.141275,1
2,EC101,1.065106,1.211953,1
3,EC102,1.052841,0.900324,1
4,EC103,0.978388,0.922686,1
5,EC104,1.042415,0.739762,1
6,EC105,1.007035,0.289287,1
7,EC106,1.012674,0.270663,1
8,EC107,1.056800,0.585636,1
9,EC108,1.060703,0.774899,1


## Perceptron Learning Algorithm

In [ ]:
# Check for linear separability: LDA
# PLA will only converege if the data is linearly separable.

#-----Hypothesis-----
#Note: Response must be +1 or -1
def hypothesis(response, predictors, w):
    
    p = predictors.shape[2]
    
    x = np.transpose(np.matrix(predictors))
    y = np.matrix(response)
    
    pred = np.transpose(np.sign(np.matmul(np.transpose(w), x)))
    error = (pred != y)  
    
    return pred, error



def PLA(response, predictors, iterations):
    N = len(response)
    p = predictors.shape[2] # num predictors
    
    x = np.transpose(np.matrix(predictors))
    y = np.matrix(response)
  
    w  = np.random.uniform(low = 0, high = 1, size = p)
    pred = hypothesis(y, x, w)

###########################################STILL NEED TO FIX    
    for i in range(0, iterations):
        if(sum(pred$error) >= 1):
            #sample a misclassified point
            mis_class = which(pred$error == 1)
            ind = sample(mis.class,1)
            
            #update its weight
            w = w + y[ind,]*x[,ind] 
            
            #refit the model
            pred = h(data, w)
        
        else: break
        
    return(w)    

## Support Vector Machine

In [429]:
# Yearly
target_1 = []

for i in range(0,len(profile)):
    
    if profile['profile'].iloc[i] == 1: 
        target_1.append(1)
    else: target_1.append(0)

          
target_2 = []        
for i in range(0,len(profile)):
    
    if profile['profile'].iloc[i] == 2: 
        target_2.append(1)
    else: target_2.append(0)

        
target_3 = []        
for i in range(0,len(profile)):
    
    if profile['profile'].iloc[i] == 3: 
        target_3.append(1)
    else: target_3.append(0)
        
        
target_4 = []        
for i in range(0,len(profile)):
    
    if profile['profile'].iloc[i] == 4: 
        target_4.append(1)
    else: target_4.append(0)


In [465]:
Data = pd.DataFrame()

Data = profile[['Unnamed: 0', 'date', 'mun_code']]

target1_series = pd.Series(target_1)
target2_series = pd.Series(target_2)
target3_series = pd.Series(target_3)
target4_series = pd.Series(target_4)

Data['target1'] = target1_series.values
Data['target2'] = target2_series.values
Data['target3'] = target3_series.values
Data['target4'] = target4_series.values
Data

/home/dside/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/dside/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/home/dside/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,Unnamed: 0,date,mun_code,target1,target2,target3,target4
0,0,2012.0,BUF,0,0,0,1
1,1,2012.0,CPT,0,0,0,1
2,46,2012.0,EC101,0,0,0,1
3,47,2012.0,EC102,0,0,0,1
4,48,2012.0,EC103,0,0,0,1
5,49,2012.0,EC104,0,0,0,1
6,50,2012.0,EC105,0,0,0,1
7,51,2012.0,EC106,0,0,0,1
8,52,2012.0,EC107,0,0,0,1
9,53,2012.0,EC108,0,0,0,1


In [466]:
Data = Data.merge(svm_predictors, on = ['mun_code'], how = 'left') #(1866, 15) should only have 933 rows
del Data['Unnamed: 0'] 
Data = Data.drop_duplicates()

In [467]:
Data
#Data.to_csv('/home/dside/Documents/GitHub/SVM_Data.csv')

,date,mun_code,target1,target2,target3,target4,Tertiary_educ,birth_prop,EmpAdult_Prop,Citizenship_Prop,EducAttend_Prop,Female_Prop,Overcrowded_Prop,IncPoverty_Prop,Living_with_Parents_Prop
0,2012.0,BUF,0,0,0,1,0.069307,0.249921,0.586196,0.985233,0.572181,0.508867,0.198961,0.556065,0.215037
2,2012.0,CPT,0,0,0,1,0.089004,0.259899,0.785328,0.954220,0.460475,0.510679,0.135318,0.373324,0.353355
4,2012.0,EC101,0,0,0,1,0.036421,0.309103,0.685474,0.992386,0.424163,0.488112,0.182157,0.615141,0.248494
6,2012.0,EC102,0,0,0,1,0.023654,0.310874,0.516252,0.989414,0.398775,0.504205,0.129970,0.659468,0.236147
8,2012.0,EC103,0,0,0,1,0.017632,0.353147,0.652574,0.983153,0.398154,0.516187,0.104969,0.693819,0.227746
10,2012.0,EC104,0,0,0,1,0.117075,0.183982,0.606591,0.974696,0.534894,0.521548,0.155457,0.559891,0.227053
12,2012.0,EC105,0,0,0,1,0.026350,0.264853,0.599131,0.979132,0.435303,0.494508,0.150628,0.619292,0.237644
14,2012.0,EC106,0,0,0,1,0.015456,0.340372,0.695891,0.961879,0.331143,0.457796,0.131357,0.635276,0.282877
16,2012.0,EC107,0,0,0,1,0.013453,0.396996,0.606949,0.991768,0.328194,0.487126,0.174465,0.677640,0.254633
18,2012.0,EC108,0,0,0,1,0.029454,0.373232,0.768660,0.977036,0.345957,0.499622,0.136771,0.530460,0.344163


## Fitting the model

In [456]:
#Data for profile 1:
profile_1 = Data
#profile_1_2015 = profile_1[profile_1['date'] == 2015]
del profile_1['target2']
del profile_1['target3']
del profile_1['target4'] 
del profile_1['date']
del profile_1['mun_code']
profile_1 = profile_1.reset_index()
profile_1.to_csv('/home/dside/Documents/GitHub/Analysis/profile1.csv')


In [460]:
#Data for profile 2:
profile_2 = Data

#profile_2_2015 = profile_2[profile_2['date'] == 2015]
del profile_2['target1']
del profile_2['target3']
del profile_2['target4'] 
del profile_2['date']
del profile_2['mun_code']
profile_2 = profile_2.reset_index()
profile_2.to_csv('/home/dside/Documents/GitHub/Analysis/profile2.csv')

#profile_2_2015

In [464]:
#Data for profile 3:
profile3 = Data

#profile_3_2015 = profile3[profile3['date'] == 2015]
del profile3['target1']
del profile3['target2']
del profile3['target4'] 
del profile3['date']
del profile3['mun_code']
profile3 = profile3.reset_index()
profile3.to_csv('/home/dside/Documents/GitHub/Analysis/profile3.csv')

#profile_3_2015

In [468]:
#Data for profile 4:
profile4 = Data

#profile4 = profile4[profile4['date'] == 2015]
del profile4['target1']
del profile4['target2']
del profile4['target3'] 
del profile4['date']
del profile4['mun_code']
profile4 = profile4.reset_index()
profile4.to_csv('/home/dside/Documents/GitHub/Analysis/profile4.csv')


In [315]:
# fitting model to entire data set - initial test
response_2015 =  profile_2_2015['target2']
predictors_2015 =  profile_2_2015
del predictors_2015['target2']

# STEP 1: ESTIMATOR
model = svm.SVC(C=1.0, kernel= 'rbf', degree = 3, 
                   shrinking=True, probability=False, tol=0.001, cache_size=200, 
                   class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', 
                   random_state=None)

In [316]:
# STEP 2: Fitting the model on the training data
svm_fit = model.fit(predictors_2015, response_2015)  
svm_fit


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [317]:
#STEP 3: Prediction
prediction2 = model.predict(predictors_2015)
#prediction2 = prediction2.values

In [294]:
# STEP 4: Prediction Accuracy
profile_2_2015 = Data[Data['date'] == 2015]
correct_pred = []

for i in range(0,len(profile_2_2015)):
    if profile_2_2015['target2'].iloc[i] == prediction2[i]:
        correct_pred.append(1)
    else: correct_pred.append(0)


In [296]:
correct_pred = pd.Series(correct_pred)
prop_correct = correct_pred.sum() / len(profile_2_2015)
prop_correct

0.9871794871794872

## Cross-Validation

Model formulation comments
- linear kernel functions seems to fit data better than radial basis function.
- Cost of 1 seems to be optimal.
- 


In [350]:
profile_4_2015.shape

(234, 11)

In [417]:
#svm model definition
model = svm.SVC(C=1.0, kernel= 'linear', degree = 3, 
                   shrinking=True, probability=False, tol=0.0001, cache_size=200, 
                   class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', 
                   random_state=None)

In [418]:
# performing cross-validation for profile 4
N = len(profile_4_2015)
steps = math.floor(N / 10)
predictions = []


for i in range(0,9):
    
    curr_test = range(i*steps, i*steps + steps - 1)
    
    train = profile_4_2015.drop(profile_4_2015.index[curr_test])
    curr_resp = train['target4']
    curr_pred =  train
    del curr_pred['target4']
                            
    test =  profile_4_2015.iloc[curr_test]
    test_resp = test['target4']
    test_pred = test
    del test_pred['target4']
                                
    #fitting SVM model to training data
    svm_fit = model.fit(curr_pred, curr_resp)
                                
    #prediction
    predict = model.predict(test_pred)
    predictions.append(predict)  # collecting predicted values


In [370]:
#predictions
#[len(x) for x in predictions[0]]

In [419]:
#fitting the extras 
test = profile_4_2015.iloc[198:234]
test_resp = test['target4']
test_pred = test
del test_pred['target4']

train = profile_4_2015.drop(profile_4_2015.index[198:234])
curr_resp = train['target4']
curr_pred =  train
del curr_pred['target4']

svm_fit = model.fit(curr_pred, curr_resp)
predict = model.predict(test_pred) 
predictions.append(predict)  # collecting predicted values

In [420]:
flat_list = [item for sublist in predictions for item in sublist]
predictions = pd.Series(flat_list)
#predictions #198 observations

In [421]:
Accuracy = []

#Accuracy
for i in range(0,len(profile_4_2015)):
    if profile_4_2015['target4'].iloc[i] == predictions[i]:
        Accuracy.append(1)
    else: Accuracy.append(0)

#flat_Acc = [item for sublist in Accu for item in sublist]
#Accuracy
 

In [422]:
Accuracy = pd.Series(Accuracy)
prop_correct = Accuracy.sum() / len(profile_4_2015)
prop_correct

0.7051282051282052

## Use Cross-Validation to determine arguments 

## Different approach to one-vs-rest

In [478]:
data_comb = pd.DataFrame()
data_comb = profile[['profile', 'mun_code']]
data_comb = data_comb.merge(svm_predictors, on = ['mun_code'])
del data_comb['mun_code'] 
#data_comb 

In [479]:
model = svm.SVC(C=1.0, kernel= 'linear', degree = 3, 
                   shrinking=True, probability=False, tol=0.0001, cache_size=200, 
                   class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', 
                   random_state=None)

curr_resp = data_comb['profile']
curr_pred = data_comb
del curr_pred['profile']


In [ ]:
curr_resp

In [480]:
svm_fit = model.fit(curr_pred, curr_resp)

In [481]:
#prediction
predict = model.predict(curr_pred)

## Relative variable Importance

In [ ]:
# need to relfect on the relative importance of inputs for each category.
# backward elimination

In [ ]:

# Note you may wnt to perfore cross-validation to shoose optimal parameter values. 
# C penalty term of error
# kernel function options: ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’
# if the data is linearly seperable use the linear kernel.
# The kernel converts a non-linear relationship into a linear one
# decision_function_shape: one-vs-rest (‘ovr’), one-vs-one (‘ovo’)


#steps = math.floor(N / 10)
#i = 1
#curr_test = range(i*steps, i*steps + steps - 1)
#train = profile_4_2015.drop(profile_4_2015.index[curr_test])
#test =  profile_4_2015.iloc[curr_test]
#test.shape
#list(curr_Range)

#steps = math.floor(N / 10)
#i = 1
#curr_test = range(i*steps, i*steps + steps - 1)
#train = profile_4_2015.drop(profile_4_2015.index[curr_test]